# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [54]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [55]:


import gmaps.datasets
city_data = pd.read_csv("output_data/cities.csv")
gmaps.configure(api_key=g_key) # Fill in with your API key


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [56]:
locations = city_data[['Lat', 'Lng']]
weights = city_data['Humidity']
maxintensity=max(city_data['Humidity'])
pointradius=10

In [58]:
fig = gmaps.figure(center=(35, 0.1),zoom_level=1.5)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, max_intensity=maxintensity, point_radius=7))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [59]:
#  Narrow down criteria
# * A max temperature lower than 80 degrees but higher than 70.
#  * Wind speed less than 10 mph.
#  * Zero cloudiness.
    
Max_Temp_80_df=city_data.loc[city_data['Max Temp(F)'] < 80]
Min_Temp_70_df=Max_Temp_80_df.loc[Max_Temp_80_df['Max Temp(F)'] > 70]
wind_speed_df=Min_Temp_70_df.loc[Min_Temp_70_df['Wind Speed'] < 10]
Cloudiness_df=wind_speed_df.loc[wind_speed_df['Cloudiness'] == 0]


In [60]:
# Removing null records
Cloudiness_df=Cloudiness_df.dropna()
Cloudiness_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Max Temp(F)
74,Nālūt,0,LY,1586679948,17,30.33,10.85,297.87,4.59,76.496
138,Zeerust,0,ZA,1586679982,44,-25.54,26.08,298.15,3.60,77.000
174,Adrar,0,MR,1586680004,24,20.50,-10.07,298.82,6.25,78.206
202,Atar,0,MR,1586680020,25,20.52,-13.05,299.49,6.11,79.412
312,Otse,0,BW,1586680077,36,-25.02,25.73,299.15,0.50,78.800
350,Yigou,0,CN,1586680096,19,35.81,114.32,295.39,5.38,72.032
491,Aswān,0,EG,1586680165,15,24.09,32.91,295.15,5.70,71.600
526,Rustenburg,0,ZA,1586680185,20,-25.67,27.24,299.66,3.89,79.718
548,Kashgar,0,CN,1586680196,8,39.45,75.98,297.15,2.00,75.200


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [61]:
hotel_df=Cloudiness_df
hotel_df=hotel_df.reset_index(drop=True)
hotel_df["Hotel Name"]=""
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
import pandas as pd

hotel_name_df=[]

for i in range(len(hotel_df)):
    try:
        tmp_df=hotel_df.loc[i,:]
        city=tmp_df["City"]
        lat=tmp_df["Lat"]
        lng=tmp_df["Lng"]
        locations = f"{lat},{lng}"
        params = {
               "location":locations ,
               "radius": 5000,
               "type":"lodging",
               "key": g_key}
        response = requests.get(base_url, params=params).json()
        from pprint import pprint
        hotel_name_df.append(response["results"][0]["name"])
    except:
        print(f"Seems no lodging exists for the city in the 5000m radius- {city}")
        hotel_name_df.append("NaN")
        
hotel_df["Hotel Name"]=hotel_name_df
narrowed_city_df=hotel_df
narrowed_city_df

Seems no lodging exists for the city in the 5000m radius- Nālūt
Seems no lodging exists for the city in the 5000m radius- Adrar
Seems no lodging exists for the city in the 5000m radius- Otse


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Max Temp(F),Hotel Name
0,Nālūt,0,LY,1586679948,17,30.33,10.85,297.87,4.59,76.496,NaN
1,Zeerust,0,ZA,1586679982,44,-25.54,26.08,298.15,3.60,77.000,Kareespruit Game Ranch and Guest House
2,Adrar,0,MR,1586680004,24,20.50,-10.07,298.82,6.25,78.206,NaN
3,Atar,0,MR,1586680020,25,20.52,-13.05,299.49,6.11,79.412,Oumou Elghoura
4,Otse,0,BW,1586680077,36,-25.02,25.73,299.15,0.50,78.800,NaN
5,Yigou,0,CN,1586680096,19,35.81,114.32,295.39,5.38,72.032,Xinxin Restaurant
6,Aswān,0,EG,1586680165,15,24.09,32.91,295.15,5.70,71.600,Nuba Nile
7,Rustenburg,0,ZA,1586680185,20,-25.67,27.24,299.66,3.89,79.718,Rustenburg Boutique Hotel
8,Kashgar,0,CN,1586680196,8,39.45,75.98,297.15,2.00,75.200,帕米尔青年旅舍


In [62]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [63]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content= hotel_info)

fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))